# Encontro 03: Grafos Reais

Importando a biblioteca:

In [4]:
import sys
sys.path.append('..')

import socnet as sn

Carregando e visualizando o grafo:

In [6]:

sn.node_size = 3
sn.node_color = (0, 0, 0)
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

g = sn.load_graph('mySpotify.gml')

sn.show_graph(g, nlab=True)

Dependendo do tamanho, o grafo pode demorar um pouco para aparecer.